In [1]:
#!/usr/bin/env python
# coding: utf-8

# ### Get OSI SAF SIC at Hausgarten stations

# #### Modules and functions

# In[3]:


import numpy as np
import pylab as plt
import xarray as xr
import os,sys
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pyproj
import pandas as pd
import datetime
import glob

In [2]:
# In[4]:


def convert_to_decimal_degrees(coord_str):
    # Remove the degree symbol and split the string
    degree_str, minute_str_with_direction = coord_str.split('°')
    minute_str, direction = minute_str_with_direction.split("'")

    # Remove any commas from the minute string and convert to float
    minutes = float(minute_str.replace(',', '.'))

    # Convert degree string to float
    degrees = float(degree_str)

    # Calculate decimal degrees
    decimal_degrees = degrees + minutes / 60

    # If the direction is 'S' or 'W', make the decimal degrees negative
    if direction.strip() in ['S', 'W']:
        decimal_degrees = -decimal_degrees

    return decimal_degrees

In [3]:
def xydist(x,y,x_2d,y_2d):
    return np.sqrt((x-x_2d)**2 + (y-y_2d)**2)

In [4]:
def XY_To_LatLon(x,y,projstring):
    p1=pyproj.Proj(projstring,preserve_units=True)
    (lat,lon)=p1(x,y,inverse=True,)
    return(lat,lon)

In [5]:
def LatLon_To_XY(lat,lon,projstring):
    p1=pyproj.Proj(projstring,preserve_units=True)
    (x,y)=p1(lat,lon,inverse=False,)
    return(x,y)

In [6]:
def get_coorddicts(df):
    londict = {stationname:np.round(lon,3) for stationname,lon in zip(df["Station ID"],df["lon_decimal"])}
    latdict = {stationname:np.round(lat,3) for stationname,lat in zip(df["Station ID"],df["lat_decimal"])}
    return londict,latdict

In [7]:
def design_map(ax,title,hemisphere):
    if hemisphere == "nh":
        ax.set_extent([-30, 30, 75,85], ccrs.PlateCarree())
    elif hemisphere == "sh":
        ax.set_extent([-180, 180, -60,-90], ccrs.PlateCarree())
    else:
        sys.exit("Hemisphere needs to be either 'nh' or 'sh'!")
    ax.set_title(title,fontsize = 14) # set title
    ax.coastlines()
    ax.gridlines()
    #ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='#F5E5A0')  # Adjust facecolor as needed
    #ax.set_facecolor('#F5F5F5')
    #ax.gridlines().label_style = {'fontsize': 14}
    #gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, alpha=0.5, linestyle='--', draw_labels=True)
    #gl.xlabel_style = {'size': 14, 'color': 'black'}
    #gl.ylabel_style = {'size': 14, 'color': 'black'}

In [19]:
datadir_sinxs = "/Users/vludwig/05_SINXS/02_DATA/03_DATABASE/03_SYMLINKS"
datadir_hausgarten = "/Users/vludwig/06_HAUSGARTEN/02_DATA/01_CSV/"
plotdir = "/albedo/home/vludwig/06_HAUSGARTEN/03_PLOTS/"

In [75]:
fns_cs2smos = list(np.sort([fn for fn in glob.glob(os.path.join(datadir_sinxs,"SINXS_ESA_NH_SIT_HYB_CS2-SMOS_*nc"))]))
fns_envisat_before_2010 = list(np.sort([fn for fn in glob.glob(os.path.join(datadir_sinxs,"SINXS_CCI_NH_SIT_SAT_ENVISAT_200*nc"))]))
fns_envisat_spring_2010 = list(np.sort([fn for fn in glob.glob(os.path.join(datadir_sinxs,"SINXS_CCI_NH_SIT_SAT_ENVISAT_20100*nc"))]))


In [77]:
fns_all = fns_cs2smos+fns_envisat_before_2010+fns_envisat_spring_2010

In [78]:
coordfile = xr.open_dataset(fns_all[0])
x_2d,y_2d = np.meshgrid(coordfile.coords["xc"],coordfile.coords["yc"])
lon_2d,lat_2d = XY_To_LatLon(x_2d,y_2d,projstring = "+proj=laea +lat_0=90 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs")

In [79]:
df_hausgarten = pd.read_excel(os.path.join(datadir_hausgarten,"average HG-Stations.xlsx"))

# Define the list of Station IDs you want to filter by
station_ids = ['EG-I', 'EG-II', 'EG-III', 'EG-IV', 'N3', 'N4', 'N5']

# Select rows where "Station ID" is in the specified list
hausgarten_ice = df_hausgarten[df_hausgarten['Station ID'].isin(station_ids)]
inds = hausgarten_ice.index.to_list()
hausgarten_ice["lat_decimal"] = [convert_to_decimal_degrees(coordstr) for coordstr in hausgarten_ice["average lat"]]
hausgarten_ice["lon_decimal"] = [convert_to_decimal_degrees(coordstr) for coordstr in hausgarten_ice["average long"]]
hausgarten_ice["color"] = plt.rcParams['axes.prop_cycle'].by_key()['color'][0:hausgarten_ice.shape[0]]

/var/folders/d6/cf68qnyn42708tqdllr3pbmjmsjrs2/T/ipykernel_93144/1332135711.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hausgarten_ice["lat_decimal"] = [convert_to_decimal_degrees(coordstr) for coordstr in hausgarten_ice["average lat"]]
/var/folders/d6/cf68qnyn42708tqdllr3pbmjmsjrs2/T/ipykernel_93144/1332135711.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hausgarten_ice["lon_decimal"] = [convert_to_decimal_degrees(coordstr) for coordstr in hausgarten_ice["average long"]]
/var/folders/d6/cf68

In [80]:
londict,latdict = get_coorddicts(hausgarten_ice)
years = list(range(1999, 2025))
months = [f"{d:02d}" for d in range(1,13)]
station_ids = hausgarten_ice["Station ID"]


In [81]:
ds_all = xr.open_mfdataset(fns_all)

In [84]:
# Create base columns: Station ID, lon, lat, sit, year, and month
for station_id in station_ids:
    print(f"### {station_id} ###")
    lon_station = londict[station_id]
    lat_station = latdict[station_id]
    x_station,y_station = LatLon_To_XY(lon_station,lat_station,projstring)
    dist_station = xydist(x_station, y_station, x_2d,y_2d)
    minind_station = np.where(dist_station == dist_station.min())
    data_station = {
        "station_id":np.repeat(station_id,len(years)*12),
        'lon': londict[station_id],
        'lat': latdict[station_id],
        'year': np.repeat(years,len(months)),
        'month': months*len(years),
        'sit_mean': np.repeat(np.nan,len(years)*12),

    }
    
    # Create the DataFrame
    df_station = pd.DataFrame(data_station)
    sit_mean_station = []
    print("Extracting station...")
    ds_point = ds_all.isel(xc =minind_station[1], yc = minind_station[0])
    sit_masked = ds_point.sea_ice_thickness.where(ds_point.sea_ice_thickness>30,np.nan)
    for year in range(1999,2025):
        #print(year)
        for month in range(1,13):
            #print(month)
            #if np.logical_and(year in ds_point.time.dt.year, month in ds_point.time.dt.month):
            sit_month = ds_point.sea_ice_thickness.sel(time=(ds_point.time.dt.year == year) & (ds_point.time.dt.month == month)).values.flatten()
            if sit_month.size>0:    
                df_station.loc[(df_station["year"] == year) & (df_station["month"] == f"{month:02}"),"sit_mean"] = sit_month
    print(f"Saving dataframe for {station_id}...")
    df_station.to_csv(os.path.join(datadir_hausgarten,f"hausgarten_sit_mean_{station_id}_{years[0]}-{years[-1]}.csv"),index = False)
    ## df_station.to_csv(os.path.join(datadir_hausgarten,f"TMP.csv"),index = False)
    #sys.exit()

### EG-IV ###
Extracting station...
Saving dataframe for EG-IV...
### EG-III ###
Extracting station...
Saving dataframe for EG-III...
### EG-II ###
Extracting station...
Saving dataframe for EG-II...
### EG-I ###
Extracting station...
Saving dataframe for EG-I...
### N3 ###
Extracting station...
Saving dataframe for N3...
### N4 ###
Extracting station...
Saving dataframe for N4...
### N5 ###
Extracting station...
Saving dataframe for N5...


In [66]:
ds_point.sea_ice_thickness.values

array([[[7.2400e-01]],

       [[5.0400e-01]],

       [[4.2200e-01]],

       [[3.7500e-01]],

       [[7.1100e-01]],

       [[4.1600e-01]],

       [[3.2767e+04]],

       [[3.2767e+04]],

       [[7.8000e-01]],

       [[7.5400e-01]],

       [[8.6100e-01]],

       [[5.7900e-01]],

       [[6.9500e-01]],

       [[2.7100e-01]],

       [[2.1000e-01]],

       [[5.3500e-01]],

       [[8.4900e-01]],

       [[4.5200e-01]],

       [[7.6100e-01]],

       [[5.4900e-01]],

       [[1.1150e+00]],

       [[1.1670e+00]],

       [[8.4100e-01]],

       [[6.5200e-01]],

       [[7.6900e-01]],

       [[2.6100e-01]],

       [[1.9600e-01]],

       [[3.2767e+04]],

       [[2.5300e-01]],

       [[4.6300e-01]],

       [[4.2100e-01]],

       [[1.0380e+00]],

       [[5.0800e-01]],

       [[2.2000e-01]],

       [[1.0550e+00]],

       [[3.2767e+04]],

       [[3.2767e+04]],

       [[3.2767e+04]],

       [[3.2767e+04]],

       [[2.2600e-01]],

       [[3.2767e+04]],

       [[2.8400e

In [50]:
sit_month

array([0.558])

In [52]:
df_station.loc[(df_station["year"] == year) & (df_station["month"] == f"{month:02}"),"sit_mean"] = sit_month

In [53]:
df_station

,station_id,lon,lat,year,month,sit_mean
0,EG-IV,-2.754,78.728,1999,01,NaN
1,EG-IV,-2.754,78.728,1999,02,0.558
2,EG-IV,-2.754,78.728,1999,03,NaN
3,EG-IV,-2.754,78.728,1999,04,NaN
4,EG-IV,-2.754,78.728,1999,05,NaN
...,...,...,...,...,...,...
307,EG-IV,-2.754,78.728,2024,08,NaN
308,EG-IV,-2.754,78.728,2024,09,NaN
309,EG-IV,-2.754,78.728,2024,10,NaN
310,EG-IV,-2.754,78.728,2024,11,NaN
